In [69]:
import os
import re
import pandas as pd
import numpy as np
from datetime import datetime
from os.path import basename

path = 'D:\Courses\資策會_大數據班_BB105\專題\IOT\IOT-Data\\'        # Dataset folder路徑
history = 20                                                        # 設定時間序列長度 / history越高，筆數越少

# Patterns
p_include = re.compile('\d{4}-\d{2}-\d{2}')                        # 只納入日期格式folder
p_labels = re.compile('labels.csv')
p_dof = re.compile('trimmed_9_dof_\d+.csv')                        # 9_dof_\d+.csv, trimmed_9_dof_\d+.csv
p_labelled_pulse = re.compile('labelled_pulse_\d+.csv')

# Pulse貼標
pulse_file = '\\pulse_%s.csv'
labelled_pulse_file = '\\labelled_pulse_%s.csv'

# Table檔案
adjusted_pulse_table = 'adjusted_pulse_table.csv'                  
series_dof_table = 'series_dof_table.csv'                          # series_trimmed_dof.csv 結合
joined_table = 'joined_table.csv'                                  # dof_table.csv & adjusted_pulse_table.csv 結合

## Pulse data 貼標

In [70]:
def addLabels(init_path):
    folder_list = getFolderList(init_path)
    for folder in folder_list:
        addLabelToFiles(folder)

def getFolderList(init_path):
    folder_list = []
    for file in os.listdir(init_path):
        if p_include.match(file) and os.path.isdir(os.path.join(init_path,file)): 
            folder_list.append(os.path.join(init_path, file))
    return folder_list
    
def addLabelToFiles(folder):
    files = os.listdir(folder)
    for file in files:
        if p_labels.match(file):
            file_path = os.path.join(folder, file)
            appendLabelAndWriteFile(file_path)

def appendLabelAndWriteFile(file_path):
    labels_df = pd.read_csv(file_path, engine='python', header=None)
    print('folder %s' %os.path.dirname(file_path).split('\\')[-1] + '......')
    for index, row in labels_df.iterrows():
        num = row[0]
        folder_path = '\\'.join(file_path.split('\\')[:-1])
        pulse_df = pd.read_csv(folder_path + pulse_file %num, engine='python')
        pulse_df['label'] = ''
        thresh_str = row[1]
        thresh_time = datetime.strptime(thresh_str, '%H:%M:%S') # thresh時間
        df = addLabelToPulse(pulse_df, thresh_time)
        df = adjustOutliers(df)
        df.to_csv(folder_path + labelled_pulse_file %num, index=None) # 產生 labelled_pulse.csv
        print(folder_path + labelled_pulse_file %num + ' labels added')
    print('folder %s' %os.path.dirname(file_path).split('\\')[-1] + ' finish!\n') 

def addLabelToPulse(pulse_df, thresh_time):
    label_list = []
    for index, row in pulse_df.iterrows():
        pulse_time = datetime.strptime(row['datetime'].split(' ')[1], '%H:%M:%S') # pulse時間
        if pulse_time >= thresh_time:
            label_list.append(1)
        else:
            label_list.append(0)
    pulse_df['label'] = pd.Series(label_list, index=pulse_df.index)
    return pulse_df

def adjustOutliers(df):
    zeros_removed = 0
    def adjustBPM(bpm):
        if bpm == 0:
            return np.nan
        elif bpm < 60:
            return 60
        elif bpm > 190:
            return 190
        else: 
            return bpm
    df['BPM'] = df['BPM'].map(adjustBPM).fillna(method='ffill')
    before = len(df)
    df = df.dropna()
    after = len(df)
    print('NaN removed: ' + str(before-after))
    zeros_removed += (before-after)
    return df

In [71]:
addLabels(path)

folder 2018-02-28......
NaN removed: 3
D:\Courses\資策會_大數據班_BB105\專題\IOT\IOT-Data\2018-02-28\labelled_pulse_1.csv labels added
NaN removed: 2
D:\Courses\資策會_大數據班_BB105\專題\IOT\IOT-Data\2018-02-28\labelled_pulse_2.csv labels added
NaN removed: 1
D:\Courses\資策會_大數據班_BB105\專題\IOT\IOT-Data\2018-02-28\labelled_pulse_3.csv labels added
NaN removed: 3
D:\Courses\資策會_大數據班_BB105\專題\IOT\IOT-Data\2018-02-28\labelled_pulse_4.csv labels added
NaN removed: 2
D:\Courses\資策會_大數據班_BB105\專題\IOT\IOT-Data\2018-02-28\labelled_pulse_5.csv labels added
folder 2018-02-28 finish!

folder 2018-03-01......
NaN removed: 2
D:\Courses\資策會_大數據班_BB105\專題\IOT\IOT-Data\2018-03-01\labelled_pulse_1.csv labels added
NaN removed: 2
D:\Courses\資策會_大數據班_BB105\專題\IOT\IOT-Data\2018-03-01\labelled_pulse_2.csv labels added
NaN removed: 3
D:\Courses\資策會_大數據班_BB105\專題\IOT\IOT-Data\2018-03-01\labelled_pulse_3.csv labels added
NaN removed: 1
D:\Courses\資策會_大數據班_BB105\專題\IOT\IOT-Data\2018-03-01\labelled_pulse_4.csv labels added
NaN rem

## Create series_dof_table.csv & adjusted_pulse_table.csv

In [72]:
def getFileList(path, p):
    files = []
    for i in os.listdir(path):
        folder_path = os.path.join(path,i)
        if os.path.isdir(folder_path):
            for file in os.listdir(folder_path):
                if p.match(file):
                    file_path = os.path.join(folder_path, file)
                    print(file_path)
                    files.append(file_path)
    return files

def createSeriesFile(history, files):
    output_files = []
    for file in files:
        output_file = os.path.dirname(file) + '\series_' + basename(file)
        dfs = []
        for i in range(history+1):
            df = pd.read_csv(file, engine='python')
            
            if i > 0:                                                  # 大於 0 代表歷史邏輯
                for j in range(i):                                     # 產生新row並塞空值
                    df.loc[-(j+1)] = pd.Series([np.nan])

                df.index = df.index + i                                # 把所有index增加 i
                df = df.sort_index()                                   # 重新排序
                df = df.iloc[:,1:]                                     # 移除 datetime

                new_columns = []                                       # 從新命名歷史資料欄位
                for column in df.columns:
                    new_columns.append(column + '(t-' + str(i) + ')')  
                df.columns = new_columns
            dfs.insert(0, df)                                          # 把歷史資料加在陣列前方
            
        df = pd.concat(dfs, axis=1)
        df = df.dropna(axis=0)
        datetime_col = df.pop('datetime')
        df.insert(0, 'datetime', datetime_col)
        df.to_csv(output_file, index=False)
        print(output_file + ' created!')
        output_files.append(output_file)
    return output_files

def appendToSingleTable(files):
    df = pd.DataFrame()
    print('\nAppending tables......')
    for file in files:
        print(file)
        df = df.append(pd.read_csv(file, engine='python'), ignore_index=True)
    return df

In [73]:
# 建置 series_dof_table.csv (Time-series加工)
print('Retrieving data from 9_dof files...')
files = getFileList(path, p_dof)
print('\nAdding time-series to data...')
series_files = createSeriesFile(history, files)
print('\nMerging time-series data to DOF table...')
df = appendToSingleTable(series_files)                                         # merge資料
df.to_csv(path + series_dof_table, index=False, sep=',', encoding='utf-8')     # 寫出檔案
print('series_dof_table.csv created!')

# 建置 adjusted_pulse_table.csv (離群值處理)
print('\nRetrieving data from labelled_pulse files...')
files = getFileList(path, p_labelled_pulse)
df = appendToSingleTable(files)                                                  # merge資料
df.to_csv(path + adjusted_pulse_table, index=False, sep=',', encoding='utf-8')   # 寫出檔案
print('adjusted_pulse_table created!')

Retrieving data from 9_dof files...
D:\Courses\資策會_大數據班_BB105\專題\IOT\IOT-Data\2018-02-28\trimmed_9_dof_1.csv
D:\Courses\資策會_大數據班_BB105\專題\IOT\IOT-Data\2018-02-28\trimmed_9_dof_2.csv
D:\Courses\資策會_大數據班_BB105\專題\IOT\IOT-Data\2018-02-28\trimmed_9_dof_3.csv
D:\Courses\資策會_大數據班_BB105\專題\IOT\IOT-Data\2018-02-28\trimmed_9_dof_4.csv
D:\Courses\資策會_大數據班_BB105\專題\IOT\IOT-Data\2018-02-28\trimmed_9_dof_5.csv
D:\Courses\資策會_大數據班_BB105\專題\IOT\IOT-Data\2018-03-01\trimmed_9_dof_1.csv
D:\Courses\資策會_大數據班_BB105\專題\IOT\IOT-Data\2018-03-01\trimmed_9_dof_3.csv
D:\Courses\資策會_大數據班_BB105\專題\IOT\IOT-Data\2018-03-01\trimmed_9_dof_4.csv
D:\Courses\資策會_大數據班_BB105\專題\IOT\IOT-Data\2018-03-01\trimmed_9_dof_5.csv
D:\Courses\資策會_大數據班_BB105\專題\IOT\IOT-Data\2018-03-01\trimmed_9_dof_6.csv
D:\Courses\資策會_大數據班_BB105\專題\IOT\IOT-Data\2018-03-01\trimmed_9_dof_7.csv
D:\Courses\資策會_大數據班_BB105\專題\IOT\IOT-Data\2018-03-01\trimmed_9_dof_8.csv
D:\Courses\資策會_大數據班_BB105\專題\IOT\IOT-Data\2018-03-02\trimmed_9_dof_2.csv
D:\Courses\資策會_

## Create join_table.csv

In [74]:
pulse_df = pd.read_csv(path + adjusted_pulse_table, engine='python')
dof_df = pd.read_csv(path + series_dof_table, engine='python')

dof_df['datetime'] = pd.to_datetime(dof_df['datetime'])     # to datetime datatype
dof_df['datetime2'] = dof_df['datetime'].dt.round('1s')     # round to nearest second & create a datetime column called 'datetime2'
pulse_df['datetime'] = pd.to_datetime(pulse_df['datetime']) # to datetime datatype

joined_df = dof_df.join(pulse_df.set_index('datetime'), on='datetime2', how='inner')
del joined_df['datetime2']
joined_df.to_csv(path + str(history) + '_' + joined_table, index=False)
print(path + str(history) + '_' + joined_table + '..... FILE CREATED!')

D:\Courses\資策會_大數據班_BB105\專題\IOT\IOT-Data\20_joined_table.csv..... FILE CREATED!
